In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('partmtlerp.csv')
df.head()

C:\Users\Diacto\AppData\Local\Temp\ipykernel_8640\513882139.py:1: DtypeWarning: Columns (5,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('partmtlerp.csv')


,PartNum,RevisionNum,MtlSeq,MtlPartNum,QtyPer,FixedQty,RelatedOperation,MfgComment,PullAsAsm,UOMCode,Type
0,004063,0,10.0,009628,1.0,False,10.0,NaN,False,EA,Mtl
1,004958,0,10.0,024428,1.0,False,10.0,NaN,False,EA,Mtl
2,005000,0,10.0,034712,1.0,False,10.0,NaN,False,EA,Mtl
3,006090,0,10.0,014044,1.0,False,10.0,NaN,False,EA,Mtl
4,006889,0,10.0,023906,1.0,False,10.0,NaN,False,EA,Mtl


In [3]:
df =df[df['PartNum'].notna()]

In [14]:
df[df['PartNum']=='083721']

,PartNum,RevisionNum,MtlSeq,MtlPartNum,QtyPer,FixedQty,RelatedOperation,MfgComment,PullAsAsm,UOMCode,Type
97346,083721,0,10.0,083548,2.0,False,10.0,NaN,True,EA,Asm
97347,083721,0,60.0,083754,2.0,False,10.0,NaN,True,EA,Asm
97348,083721,0,80.0,083741,2.0,False,10.0,NaN,True,EA,Asm
99097,083721,0,40.0,068530,2.0,False,10.0,NaN,False,EA,Mtl
99098,083721,0,50.0,067874,2.0,False,10.0,NaN,False,EA,Mtl
99099,083721,0,70.0,083742,2.0,False,10.0,NaN,False,EA,Mtl
99366,083721,0,20.0,083730,2.0,False,10.0,NaN,False,EA,Mtl
99681,083721,0,30.0,058066,2.0,False,10.0,NaN,False,EA,Mtl


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 107139 entries, 0 to 107138
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   PartNum           107139 non-null  object 
 1   RevisionNum       107130 non-null  object 
 2   MtlSeq            107139 non-null  float64
 3   MtlPartNum        107139 non-null  object 
 4   QtyPer            107139 non-null  float64
 5   FixedQty          107139 non-null  object 
 6   RelatedOperation  107139 non-null  float64
 7   MfgComment        581 non-null     object 
 8   PullAsAsm         107139 non-null  object 
 9   UOMCode           107139 non-null  object 
 10  Type              107139 non-null  object 
dtypes: float64(3), object(8)
memory usage: 9.8+ MB


In [47]:
def get_bom(parts, rev_numbers):
    result = []
    for part in parts:
        for rev_num in rev_numbers:
            result.extend(get_bom_for_part_and_rev(part, rev_num))
    return result

def get_bom_for_part_and_rev(part, revisionnum, processed_parts=set(), paths={}, top_parent=None, top_parent_rev=None, current_parent_rev=None, path=None, level=1, additional_columns=None):
    result = []
    if top_parent is None:
        top_parent = part
    if top_parent_rev is None:
        top_parent_rev = revisionnum
    if current_parent_rev is None:
        current_parent_rev = revisionnum
    if path is None:
        path = f"{top_parent}"
    if part in processed_parts:
        return []
    processed_parts.add(part)
    paths[part] = path
    material_rows = df[(df['PartNum'] == part) & (df['RevisionNum'] == revisionnum)]
    if material_rows.empty:
        return []
    for _, material_row in material_rows.iterrows():
        material = material_row['MtlPartNum']
        if additional_columns:
            additional_values = {col: material_row[col] for col in additional_columns}
        else:
            additional_values = {}
        new_path = f"{paths.get(part, '')}/{material}"
        result.append((
            level, top_parent, top_parent_rev, part, revisionnum, current_parent_rev, new_path,
            material_row['MtlSeq'], material_row['QtyPer'], material_row['FixedQty'],
            material_row['RelatedOperation'], material_row['MfgComment'],
            material_row['PullAsAsm'], material_row['UOMCode'], 'Type',
            material
        ) + tuple(additional_values.values()))
        result.extend(get_bom_for_part_and_rev(
            material, revisionnum, processed_parts, paths, top_parent, top_parent_rev, current_parent_rev,
            new_path, level + 1, additional_columns=additional_columns
        ))
    return result

parts = ['083721']
rev_numbers = ['0']
additional_columns = ['MtlSeq', 'QtyPer', 'FixedQty', 'RelatedOperation', 'MfgComment', 'PullAsAsm', 'UOMCode', 'Type']
bom = get_bom(parts, rev_numbers)
bom_df = pd.DataFrame(bom, columns=[
    'level', 'top_parent', 'top_parent_rev', 'parent', 'revisionnum', 'current_parent_rev', 'pathexplode',
    'MtlSeq', 'QtyPer', 'FixedQty', 'RelatedOperation', 'MfgComment', 'PullAsAsm', 'UOMCode', 'Type',
    'MtlPartNum'
])

In [48]:
bom_df[bom_df['MtlPartNum']=='000515']

,level,top_parent,top_parent_rev,parent,revisionnum,current_parent_rev,pathexplode,MtlSeq,QtyPer,FixedQty,RelatedOperation,MfgComment,PullAsAsm,UOMCode,Type,MtlPartNum
33,3,083721,0,033693,0,0,083721/083548/033693/000515,10.0,2.0,False,10.0,NaN,False,EA,Type,000515


In [37]:
df[(df['MtlPartNum']=='000515')  & (df['PartNum']=='033693')]

,PartNum,RevisionNum,MtlSeq,MtlPartNum,QtyPer,FixedQty,RelatedOperation,MfgComment,PullAsAsm,UOMCode,Type
97362,033693,0,10.0,000515,2.0,False,10.0,NaN,False,EA,Mtl


In [43]:
bom_df[(bom_df['MtlPartNum']=='000515') ]

,level,top_parent,top_parent_rev,parent,revisionnum,current_parent_rev,MtlSeq,QtyPer,FixedQty,RelatedOperation,MfgComment,PullAsAsm,UOMCode,Type,MtlPartNum
33,3,083721,0,033693,0,0,10.0,2.0,False,10.0,NaN,False,EA,Type,000515


In [49]:
list =bom_df['pathexplode'].str.split('/')
pd.DataFrame(bom_df.pop('pathexplode').tolist(), columns=[f'level{i}' for i in range(1, len(bom_df['pathexplode'].iloc[0]) + 1)]))

0                     [083721, 083548]
1             [083721, 083548, 080196]
2     [083721, 083548, 080196, 080197]
3     [083721, 083548, 080196, 080198]
4     [083721, 083548, 080196, 080199]
5     [083721, 083548, 080196, 080200]
6     [083721, 083548, 080196, 001315]
7     [083721, 083548, 080196, 080204]
8     [083721, 083548, 080196, 080205]
9     [083721, 083548, 080196, 080207]
10    [083721, 083548, 080196, 080273]
11    [083721, 083548, 080196, 080206]
12    [083721, 083548, 080196, 080272]
13            [083721, 083548, 083550]
14            [083721, 083548, 083522]
15            [083721, 083548, 083732]
16            [083721, 083548, 083731]
17            [083721, 083548, 068170]
18            [083721, 083548, 083557]
19            [083721, 083548, 071968]
20            [083721, 083548, 074879]
21            [083721, 083548, 033692]
22            [083721, 083548, 069076]
23            [083721, 083548, 083733]
24            [083721, 083548, 075349]
25            [083721, 08